In [1]:
import os
from tqdm import tqdm
import pandas as pd 
import numpy as np

import re
import string
from textblob import TextBlob
from bs4  import BeautifulSoup

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

#Lemmatizing and POSTagging

from nltk import pos_tag
from nltk.tag.stanford import StanfordPOSTagger
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk_lemmatizer = WordNetLemmatizer()

from sklearn import preprocessing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.preprocessing.text import Tokenizer,text_to_word_sequence
from keras.preprocessing import sequence

punct = set(string.punctuation)
modalVerb =['can', 'could', 'may', 'might', 'will', 'would', 'shall', 'should', 'must']
stopWords = set(stopwords.words('english'))
stopWords.remove('not')
stopWords.remove('can')
stopWords.remove('will')
  

tqdm.pandas() 

import pickle
from keras.models import model_from_json


Using TensorFlow backend.


In [2]:
java_path = "C:/Program Files/Java/jdk1.8.0_201/bin/java.exe"
os.environ['JAVAHOME'] = java_path
#os.environ['STANFORD_POSTAGGER'] = "C:/Users/Me/Documents/stanford-postagger-full-2018-10-16"
os.environ['STANFORD_POSTAGGER'] = "D:/NLP/Library file/stanford-postagger-full-2018-10-16"
Stanford_postagger = StanfordPOSTagger(os.environ['STANFORD_POSTAGGER'] +'/models/english-bidirectional-distsim.tagger',os.environ['STANFORD_POSTAGGER']+'/stanford-postagger.jar')

# 1. global variable

In [3]:
#IN MAIN AS WELL AS DEPLOYABLE SCRIPT
INPUT_TEXT_LENGTH = 294 

PROJ_NAME= "1.0 Glove_pretrained_100dims_20k_LSTM"
MODEL_FILEPATH = f'model_tokenizers_weights_and_json/{PROJ_NAME}_best_model.json'
WEIGHT_FILEPATH = f'model_tokenizers_weights_and_json/{PROJ_NAME}_best_weights.h5'
TOKENIZER_FILEPATH = f'model_tokenizers_weights_and_json/{PROJ_NAME}_tokenizer_instance.pickle'

 # TO DO @ Deployable stage

# 1. preprocessing - total steps

# pre process step1

In [4]:
def replace_contraction(text):
    contraction_patterns = [ (r'won\'t', 'will not'), (r'can\'t', 'can not'), (r'i\'m', 'i am'), (r'ain\'t', 'is not'),
                            (r'(\w+)\'ll', '\g<1> will'), (r'(\w+)n\'t', '\g<1> not'),(r'(\w+)\'ve', '\g<1> have'),
                            (r'(\w+)\'s', '\g<1> is'), (r'(\w+)\'re', '\g<1> are'), (r'(\w+)\'d', '\g<1> would'), 
                            (r'&', 'and'), (r'dammit', 'damn it'), (r'dont', 'do not'), (r'wont', 'will not'),
                            (r'br', ' '),(r'www', ' '),
                            (r'http\S+', ' '),(r"#(\w+)", ' '),(r"@(\w+)", ' '),(r'[^\w\s]', ' ')                            
                           ]

    patterns = [(re.compile(regex), repl) for (regex, repl) in contraction_patterns]
    for (pattern, repl) in patterns:
        (text, count) = re.subn(pattern, repl, text)
    return text

#can run spell checker
# before punct removal of https <br> /n etc ..
# not removed stop word
#contradiction handling from below link
# pos tag
#lemmatization
#find max_length to detect freq distribution

def preProcess(text):
    
    soup = BeautifulSoup(text)# also added 'html.parser' as an argument to constructor
    text =soup.get_text()
    
    tokens = word_tokenize(text) 
    #lowercase
    tokens = [t.lower() for t in tokens]                                     #incas if its U.S untd state, IFB
    # rmv that are not alphabet
    tokens = [word for word in tokens if not word.isdigit()]
    #rmv token that are of length 1
    tokens = [word for word in tokens if len(word) > 1]
    #rmv stopword
    tokens = [word for word in tokens if word not in stopWords]
    
    return ' '.join(text for text in tokens if text not in punct)



In [5]:
#preProcess(replace_contraction("This movie really sucks! Can I get my money back please?"))

# pre process step2

In [6]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None 
    
#tagged is defined globally

def nltk_lemma(tag):
    global tagged
    lst=[]
    for word, tag in tagged:
        
        wntag = get_wordnet_pos(tag)
        
        if wntag is None:# not supply tag in case of None
            lemma = nltk_lemmatizer.lemmatize(word)
            lst.append(lemma)
            #return " ".join(lemma)
            #print(lemma)
        else:
            lemma = nltk_lemmatizer.lemmatize(word, pos=wntag)
            lst.append(lemma)
            #return " ".join(lemma)
            #print(lemma)
            
    return lst


def POS_lemmatize(review):
    global tagged
    #normal tagger
    tagged = pos_tag(word_tokenize(review))
    #stanford tagger
    #tagged = Stanford_postagger.tag(word_tokenize(review))# imp word tokenize and repective POS
    return " ".join(nltk_lemma(tagged))# retuns setence after lemmatising

In [7]:
#POS_lemmatize(val)

# 2. Importing tokenizer & checking Vectorizing new inputs 
* store tokenizer @ training and retrive its values @ deployment 
* its working illustration 5.mlmastery @@ Vectorising KERAS - GRU.ipynb


In [8]:
'''its working illustration 5.mlmastery @@ Vectorising KERAS - GRU.ipynb'''
# in deployement script    
#iniitialize it in the deployement script along with the .pickle file @ specified location
with open(TOKENIZER_FILEPATH, 'rb') as handle:
    prod_instance_tokenizer = pickle.load(handle)

print(type(prod_instance_tokenizer))

<class 'keras_preprocessing.text.Tokenizer'>


In [60]:
prod_instance_tokenizer.word_index

{'movie': 1,
 'film': 2,
 'one': 3,
 'make': 4,
 'like': 5,
 'see': 6,
 'get': 7,
 'good': 8,
 'time': 9,
 'would': 10,
 'character': 11,
 'well': 12,
 'go': 13,
 'watch': 14,
 'even': 15,
 'story': 16,
 'bad': 17,
 'really': 18,
 'think': 19,
 'show': 20,
 'scene': 21,
 'great': 22,
 'look': 23,
 'much': 24,
 'say': 25,
 'end': 26,
 'could': 27,
 'know': 28,
 'people': 29,
 'also': 30,
 'first': 31,
 'give': 32,
 'take': 33,
 'way': 34,
 'love': 35,
 'act': 36,
 'thing': 37,
 'come': 38,
 'play': 39,
 'find': 40,
 'life': 41,
 'seem': 42,
 'plot': 43,
 'work': 44,
 'actor': 45,
 'two': 46,
 'many': 47,
 'year': 48,
 'never': 49,
 'want': 50,
 'best': 51,
 'little': 52,
 'try': 53,
 'ever': 54,
 'man': 55,
 'still': 56,
 'part': 57,
 'something': 58,
 'director': 59,
 'old': 60,
 'back': 61,
 'use': 62,
 'funny': 63,
 'lot': 64,
 'real': 65,
 'guy': 66,
 'performance': 67,
 'woman': 68,
 'though': 69,
 'feel': 70,
 'star': 71,
 'another': 72,
 'big': 73,
 'cast': 74,
 'actually': 75,
 

In [125]:

def convertSentToVect(txtInput):
    inputTxt = preProcess(replace_contraction(txtInput))    
    # This is IMP KERAS text_to_word_sequence 
    inputTxt = text_to_word_sequence(POS_lemmatize(txtInput)) 
    #print(inputTxt)
    
    word_to_int= prod_instance_tokenizer.texts_to_sequences(inputTxt) # mapping it to int value from vocab    
    seq =  [sum(word_to_int,[])] 
    #print(seq)
    #ONLY PADDING IS REMAINIG  
    padded = sequence.pad_sequences(seq , maxlen=INPUT_TEXT_LENGTH,padding='pre', truncating='pre')  
    return padded
                          

# call

In [65]:
'''txtInput = "They didn’t fit either. Straight high sticks at the end. On par with other buds I have. Lesson learned to avoid."
val = preProcessUserInput(txtInput)'''

'txtInput = "They didn’t fit either. Straight high sticks at the end. On par with other buds I have. Lesson learned to avoid."\nval = preProcessUserInput(txtInput)'

In [11]:
val.shape

(1, 294)

# 3. Contruct model and weghits for prediction                          

In [12]:
'''
#used only if we export seperatly
def init(): 
	json_file = open(MODEL_FILEPATH,'r')
	loaded_model_json = json_file.read()
	json_file.close()
	loaded_model = model_from_json(loaded_model_json)
    
    
	#load woeights into new model
	loaded_model.load_weights(WEIGHT_FILEPATH)
	print("Loaded Weight from disk")

	#compile and evaluate loaded model
	loaded_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
	#loss,accuracy = model.evaluate(X_test,y_test)
	#print('loss:', loss)
	#print('accuracy:', accuracy)
	graph = tf.get_default_graph()

	return loaded_model,graph
'''
#used this coz we used Model Checkpointing
from keras.models import load_model
new_model = load_model(MODEL_FILEPATH)
## On web it expects beow line
## new_model._make_predict_function()

In [134]:
def perform_prediction(txtInput):
        val = convertSentToVect(txtInput)
        
        #sentiment = new_model.predict_classes(val)
        sentiment = new_model.predict_proba(val)
        print(sentiment)
        '''
        if sentiment[0] == 1:
            print("positive")
           
        elif  sentiment[0] == 0:  
            print("negative")
          
        else:
            print('ss',sentiment)
        '''
        

In [135]:
# 1 is pos 

text1 = "This movie is fantastic! I really like it because it is so good!"
text2 = "Good movie!"
text3 = "Maybe I like this movie."
text4 = "Meh ..."
text5 = "If I were a drunk teenager then this movie might be good."
text6 = "Bad movie!"
text7 = "Not a good movie!"
text8 = "This movie really sucks! Can I get my money back please?"
text9 = "No scenario, bad actors Give a such budget to make this... In Belgium, we make ten films which win all prices in Cannes with this.<br /><br />Last time that I've seen a such NULL-Film was Hypercube. But scenario was better.<br /><br />Is anyone knows if the director was a graduate in school-film or a cop ?<br /><br />The better things in this film was the word Why authorize to sell this ? 1Ã§ is to expensive. <br /><br />I've pay ten dollars to buy this.<br /><br />For me, pay for this was my BIG MISTAKE of millennium.<br /><br />Too bad.<br /><br />Next time I'll break my arm but buy this type of sh*t"
text10 = "Wealthy horse ranchers in Buenos Aires have a long-standing no-trading policy with the Crawfords of Manhattan, but what happens when the mustachioed Latin son falls for a certain Crawford with bright eyes, blonde hair, and some perky moves on the dance floor? 20th Century-Fox musical has a glossy veneer yet seems a bit tatty around the edges. It is very heavy on the frenetic, gymnastic-like dancing, exceedingly thin on story. Betty Grable (an eleventh hour replacement for Alice Faye) gives it a boost, even though she's paired with leaden Don Ameche (in tan make-up and slick hair). Also good: Charlotte Greenwood as Betty's pithy aunt, a limousine driver who's constantly asleep on the job, and Carmen Miranda playing herself (who else?). The stock shots of Argentina far outclass the action filmed on the Fox backlot, and some of the supporting performances are quite awful. By the time of the big horserace finale, most viewers will have had enough."
text11 = "The package received was blank and has no barcode. A waste of time and money."
texts = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10, text11]

for i, val in enumerate(texts):
    perform_prediction(val)


[[0.9157539]]
[[0.65572906]]
[[0.48763373]]
[[0.58145475]]
[[0.51110476]]
[[0.10963217]]
[[0.65572906]]
[[0.49974877]]
[[0.08980442]]
[[0.47798303]]
[[0.03558333]]


In [113]:
seq = prod_instance_tokenizer.texts_to_sequences(text_to_word_sequence("movie ups downs good stuff movie much outweighs bad good movie indeed sometimes dialogue sound light one noticed way set light amateur act good highly original storyline intense atmosphere gore factor high effect do supremely definitely worth watch maybe even must see horror gore fan"))

In [91]:
print(sum(seq,[]))

[1, 1516, 8, 392, 1, 24, 17, 8, 1, 706, 414, 294, 186, 313, 3, 4703, 34, 89, 313, 1865, 36, 8, 423, 117, 623, 1311, 694, 461, 1567, 168, 139, 125, 7979, 312, 183, 14, 179, 15, 111, 6, 90, 461, 106]
